In [2]:
import xarray as xr, pandas as pd, numpy as np
from statsmodels.nonparametric.smoothers_lowess import lowess

import cartopy, regionmask, geopandas as gpd
from shapely.geometry import Polygon

import glob, re
from matplotlib import pyplot as plt


# method to wrap longitude from (0,360) to (-180,180)
def wrap_lon(ds):
    
    if "longitude" in ds.coords:
        lon = "longitude"
        lat = "latitude"
    elif "lon" in ds.coords:
        lon = "lon"
        lat = "lat"
    else: 
        # can only wrap longitude
        return ds
    
    if ds[lon].max() > 180:
        ds[lon] = (ds[lon].dims, (((ds[lon].values + 180) % 360) - 180), ds[lon].attrs)
        
    if lon in ds.dims:
        ds = ds.reindex({ lon : np.sort(ds[lon]) })
        ds = ds.reindex({ lat : np.sort(ds[lat]) })
    return ds

ERROR 1: PROJ: proj_create_from_database: Open of /home/cb2714/conda/envs/wwa/share/proj failed


In [9]:
da = wrap_lon(xr.open_dataset("era5_t2m_daily_af_34-42E_-5-5N_-999-lastyear_su.nc").t2m)
lsm = wrap_lon(xr.open_dataset("lsmask_era5_t2m_daily_af_34-42E_-5-5N_-999-lastyear_su.nc")).lsm

In [10]:
da = da.where(lsm >= 0.5)

<xarray.DataArray 't2m' (time: 1, lat: 41, lon: 33)> Size: 5kB
array([[[23.149292, 22.531921, 21.510681, ...,       nan,       nan,
               nan],
        [22.824738, 22.075378, 21.404327, ...,       nan,       nan,
               nan],
        [22.641968, 21.600769, 20.958038, ...,       nan,       nan,
               nan],
        ...,
        [27.31189 , 28.352814, 29.474304, ..., 26.314392, 28.281189,
         29.283722],
        [28.06546 , 29.036072, 29.70633 , ..., 26.959259, 27.88861 ,
         28.897491],
        [28.369568, 29.060333, 29.532593, ..., 26.854126, 27.66034 ,
         28.485474]]], shape=(1, 41, 33), dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 8B 1979-01-01
  * lon      (lon) float32 132B 34.0 34.25 34.5 34.75 ... 41.25 41.5 41.75 42.0
  * lat      (lat) float32 164B -5.0 -4.75 -4.5 -4.25 -4.0 ... 4.25 4.5 4.75 5.0
Attributes:
    long_name:  Near-Surface Air Temperature
    units:      Celsius